In [11]:
from helper import euclidean_distance
import pandas as pd

def knn_linear_search(dataset: pd.DataFrame, target_id: int, k: int):

    POI = dataset.loc[dataset['@id'] == target_id].iloc[0]

    dataset['@dist'] = dataset.apply(lambda row: euclidean_distance(row, POI), axis=1)

    top_k = dataset.nsmallest(k+1, '@dist', keep='first')

    result = top_k[1:][['@id', '@dist']].values.tolist()

    return result

In [12]:
df = pd.read_csv('../dataset/clean_nyc_dataset.csv')
res = knn_linear_search(df, 1473070, 7)
print(res)

[[60966879.0, 0.0005561807080380005], [6289125422.0, 0.0006834002121722835], [1472576.0, 0.0006984410426108052], [6289126393.0, 0.0007019880412038678], [25603734.0, 0.0017121146018879187], [1718695464.0, 0.001778855615276942], [10553098184.0, 0.0017935517946225787]]
